<a href="https://colab.research.google.com/github/mattiapocci/poc_detection/blob/main/poc_dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Mount Drive

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

Mounted at /content/gdrive


#Python packages installation

In [ ]:
!pip install beautifulsoup4
!pip install pandas

# Imports

In [2]:
import pandas as pd
from bs4 import BeautifulSoup
import requests

# Load from drive

In [ ]:
%cd gdrive/MyDrive/poc_detection/
!ls
mitre_df = pd.read_csv("cleaned_mitre.csv")
print("Loaded mitre_df")
df_exploits = pd.read_csv("exploit-database/files_exploits.csv")
print("Loaded df_exploits")
df_shellcodes = pd.read_csv("exploit-database/files_shellcodes.csv")
print("Loaded df_shellcodes")
cve_exploit_df = pd.read_csv("exploit_db_cve.csv")
print("Loaded cve_exploit_df")

In [ ]:
mitre_df

In [ ]:
df_exploits

In [ ]:
df_shellcodes

In [ ]:
cve_exploit_df

# Mitre processing

## Navigate to correct directory

In [ ]:
#!git clone https://github.com/offensive-security/exploitd
#!ls gdrive/MyDrive/poc_detection/
%cd gdrive/MyDrive/poc_detection/
!ls

In [ ]:
mitre = open("allitems.xml", "r")
soup = BeautifulSoup(mitre)
#print(soup.prettify())
result = soup.prettify().splitlines()
print('\n'.join(result[:1000] + result[-1000:]))

In [ ]:
elements = soup.find_all("item")
'''
for e in elements:
    print(e)
    print("_____________________________________________-")
'''
#print(elements)

In [ ]:
'''
for e in elements:
    print(e.find("desc").string)
    try:
        print(e["name"] + " | " + e["seq"] + " | " + e["type"])
    except:
        print("invalid entry")
'''
mitre_df = pd.DataFrame(columns=["name", "type", "desc"])
list_of_lists = []
for e in elements:
    list_of_lists.append([str(e["name"]), str(e["type"]), e.find("desc").string])
    '''
    mitre_df.append({
        "name": e.find("name").string,
        "type":  e.find("type").string,
        "desc": e.find("desc").string
      }, ignore_index=True)
    '''
    
mitre_df = pd.DataFrame(list_of_lists, columns=['name', 'type', 'desc'])
mitre_df

### Processing

In [ ]:

print("1, initial", mitre_df.shape, "TOT = ", mitre_df.shape[0]*mitre_df.shape[1])

print("__________________________________________")
for e in elements:
    print(e.find("desc").string)
#mitre_df = mitre_df.drop(mitre_df[mitre_df["type"] == "CAN"].index)
#mitre_df=mitre_df.drop(columns=["type"])
patternDel = "^\*\*"
filter = mitre_df["desc"].str.contains(patternDel)
mitre_df = mitre_df[~filter]

print("2, descrizione senza **", mitre_df.shape, "TOT = ", mitre_df.shape[0]*mitre_df.shape[1])
mitre_df

### Load csv

In [ ]:
!ls
mitre_df.to_csv(r'cleaned_mitre.csv', index = False)
#mitre_df = pd.read_csv("cleaned_mitre.csv")
mitre_df

# Exploit db

In [ ]:
#df = pd.read_csv("/exploitdb/", encoding='ISO-8859-1')
!ls
#%cd exploit-database/
#!ls

### Load files_exploits and files_shellcodes

In [ ]:
files_exploits = open("files_exploits.csv", 'r')
files_shellcodes = open("files_shellcodes.csv", 'r')
df_exploits = pd.read_csv(files_exploits)
df_shellcodes = pd.read_csv(files_shellcodes)
print("Exploits\n")
df_exploits
#print("Shellcodes\n")
#df_shellcodes

In [ ]:
print("Shellcodes\n")
df_shellcodes

# Mapping CVE and exploit-db

In [3]:
mapping_url = 'https://cve.mitre.org/data/refs/refmap/source-EXPLOIT-DB.html'
mapping_site = requests.get(mapping_url)
soup = BeautifulSoup(mapping_site.text)
print(soup.prettify())

ame=CVE-2009-3349">
      CVE-2009-3349
     </a>
    </td>
   </tr>
   <tr>
    <td>
     EXPLOIT-DB:9646
    </td>
    <td>
     <a href="http://cve.mitre.org/cgi-bin/cvename.cgi?name=CVE-2009-3322">
      CVE-2009-3322
     </a>
    </td>
   </tr>
   <tr>
    <td>
     EXPLOIT-DB:9647
    </td>
    <td>
     <a href="http://cve.mitre.org/cgi-bin/cvename.cgi?name=CVE-2009-3361">
      CVE-2009-3361
     </a>
    </td>
   </tr>
   <tr>
    <td>
     EXPLOIT-DB:9648
    </td>
    <td>
     <a href="http://cve.mitre.org/cgi-bin/cvename.cgi?name=CVE-2009-3357">
      CVE-2009-3357
     </a>
     <a href="http://cve.mitre.org/cgi-bin/cvename.cgi?name=CVE-2009-3368">
      CVE-2009-3368
     </a>
    </td>
   </tr>
   <tr>
    <td>
     EXPLOIT-DB:9649
    </td>
    <td>
     <a href="http://cve.mitre.org/cgi-bin/cvename.cgi?name=CVE-2009-3544">
      CVE-2009-3544
     </a>
    </td>
   </tr>
   <tr>
    <td>
     EXPLOIT-DB:9653
    </td>
    <td>
     <a href="http://cve.mitre.org/cgi-b

### Create cve_exploit_df, a dataframe with exploit_db_id and cve_id for mapping

In [4]:
cve_exploit_df = pd.DataFrame(columns=["exploit_db_id", "cve_id"])
tr_list = soup.find_all('tr')
list_of_lists = []
for tr in tr_list:
    td_list = tr.find_all('td')
    
    try:
        list_of_lists.append([str(td_list[0].string), str(td_list[1].text)])
        #cve_exploit_df.append(pd.DataFrame([str(td_list[0].string), str(td_list[1].text)]))
    except:
        pass
    #print(tr)
    
cve_exploit_df = pd.DataFrame(list_of_lists, columns=["exploit_db_id","cve_id"])
print('success')


success


In [5]:
cve_exploit_df

,exploit_db_id,cve_id
0,None,\n\n\nGo to for:\nCVSS ScoresCPE Info\n\n\n\n\...
1,None,
2,Source,EXPLOIT-DB
3,Description,Exploits Database
4,URL,http://www.exploit-db.com
...,...,...
10839,EXPLOIT-DB:9734,CVE-2009-4661\n
10840,EXPLOIT-DB:983,CVE-2005-1667\n
10841,EXPLOIT-DB:9926,CVE-2011-4908\n
10842,EXPLOIT-DB:Exploit Database,CVE-2011-4094 CVE-2014-1860 CVE-2014-2072 CVE...


### Filter 

In [6]:
print("__________________________________________")
#levo quelli senza pattern su exploit_db_id
patternDel = "^EXPLOIT-DB"
filter = cve_exploit_df["exploit_db_id"].str.contains(patternDel)
cve_exploit_df = cve_exploit_df[filter]
cve_exploit_df

__________________________________________


,exploit_db_id,cve_id
6,EXPLOIT-DB:10102,CVE-2009-4186\n
7,EXPLOIT-DB:1013,CVE-2005-1598\n
8,EXPLOIT-DB:10168,CVE-2009-4767\n
9,EXPLOIT-DB:10180,CVE-2009-4091 CVE-2009-4092 CVE-2009-4093\n
10,EXPLOIT-DB:10183,CVE-2011-4906\n
...,...,...
10839,EXPLOIT-DB:9734,CVE-2009-4661\n
10840,EXPLOIT-DB:983,CVE-2005-1667\n
10841,EXPLOIT-DB:9926,CVE-2011-4908\n
10842,EXPLOIT-DB:Exploit Database,CVE-2011-4094 CVE-2014-1860 CVE-2014-2072 CVE...


In [7]:
for index, row in cve_exploit_df.iterrows():
    exploit_db_id = row['exploit_db_id']
    try:
        id = exploit_db_id.split(':')[1]
        cve_exploit_df.loc[index, 'exploit_db_id'] = id
    except:
        cve_exploit_df.loc[index, 'exploit_db_id'] = "Invalid"

cve_exploit_df

C:\Users\matti\AppData\Local\Programs\Python\Python39\lib\site-packages\pandas\core\indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)
C:\Users\matti\AppData\Local\Programs\Python\Python39\lib\site-packages\pandas\core\indexing.py:692: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value, self.name)


,exploit_db_id,cve_id
6,10102,CVE-2009-4186\n
7,1013,CVE-2005-1598\n
8,10168,CVE-2009-4767\n
9,10180,CVE-2009-4091 CVE-2009-4092 CVE-2009-4093\n
10,10183,CVE-2011-4906\n
...,...,...
10839,9734,CVE-2009-4661\n
10840,983,CVE-2005-1667\n
10841,9926,CVE-2011-4908\n
10842,Exploit Database,CVE-2011-4094 CVE-2014-1860 CVE-2014-2072 CVE...


In [8]:
for index, row in cve_exploit_df.iterrows():
    cve_id = row['cve_id']
    try:
        id = cve_id.split('\n')[0]
        cve_exploit_df.loc[index, 'cve_id'] = id
    except:
        cve_exploit_df.loc[index, 'cve_id'] = "Invalid"

cve_exploit_df

C:\Users\matti\AppData\Local\Programs\Python\Python39\lib\site-packages\pandas\core\indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)
C:\Users\matti\AppData\Local\Programs\Python\Python39\lib\site-packages\pandas\core\indexing.py:692: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value, self.name)


,exploit_db_id,cve_id
6,10102,CVE-2009-4186
7,1013,CVE-2005-1598
8,10168,CVE-2009-4767
9,10180,CVE-2009-4091 CVE-2009-4092 CVE-2009-4093
10,10183,CVE-2011-4906
...,...,...
10839,9734,CVE-2009-4661
10840,983,CVE-2005-1667
10841,9926,CVE-2011-4908
10842,Exploit Database,CVE-2011-4094 CVE-2014-1860 CVE-2014-2072 CVE...


## Save/**load**

In [10]:
cve_exploit_df.to_csv(r'datasets/exploit_db_cve.csv', index = False)
#cve_exploit_df = pd.read_csv("datasets/exploit_db_cve.csv")
# !ls


# CVE clustering

In [ ]:
#missing values
#remove stop words
#remove punctuation
#remove adjectives, adverbs, verbs
#remove version numbers
#remove single character words
df = pd.read_csv("cleaned_mitre.csv")
df

In [ ]:
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
stop = stopwords.words('english')

In [ ]:
print(stop)

### Remove stop words from df

In [ ]:
df['desc'] = df['desc'].apply(lambda x: x.lower())
#df['desc'] = df['desc'].apply(lambda x: (word for word in x if word not in stop))
#df['desc'] = df['desc'].apply(lambda x: [item for item in x if item not in stop])
df['desc'] = df['desc'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))
df['desc'][0]

In [ ]:
df1 = df
df1

In [ ]:
#remove everything that doesnt contain windows
patternDel = ".*windows.*"
filter = df["desc"].str.contains(patternDel)
df = df[filter]
df = df.reset_index()
df

In [ ]:
df['desc'][11]
df1 = df
#df = df1

In [ ]:
#remove single char words
df['desc'] = df['desc'].apply(lambda x: ' '.join([word for word in x.split() if len(word) != 1]))
df


### Spacy

In [ ]:
import spacy

nlp = spacy.load("en_core_web_sm")
doc = nlp("ip_input.c bsd-derived tcp/ip implementations allows remote attackers cause denial service (crash hang) via crafted packets.")
for token in doc:
    print(token.text, token.pos_, token.dep_)

# Nvd CVE

In [ ]:
import json
import pprint

f = open('nvdcve-1.1-modified.json','r')
nvd = json.load(f)
print(nvd.keys())
print(len(nvd['CVE_Items']))
print((nvd['CVE_Items'])[0].keys())
print((nvd['CVE_Items'])[0]['impact'])
print((nvd['CVE_Items'])[0]['cve'].keys())
print((nvd['CVE_Items'])[0]['cve']['CVE_data_meta']['ID'])
print((nvd['CVE_Items'])[0]['cve']['references']['reference_data'])
print((nvd['CVE_Items'])[0]['cve']['description']['description_data'][0]['value'])

In [ ]:
final_dict = {}
for cve in nvd['CVE_Items']:
    final_dict[cve['cve']['CVE_data_meta']['ID']] = {}
    final_dict[cve['cve']['CVE_data_meta']['ID']]['references'] = cve['cve']['references']['reference_data']
    final_dict[cve['cve']['CVE_data_meta']['ID']]['description'] = cve['cve']['description']['description_data'][0]['value']

In [ ]:
print(final_dict)

In [ ]:
with open("nvd.json", "w") as outfile:  
    json.dump(final_dict, outfile) 

In [ ]:
!ls

# Clone git of CVEProject/cvelist

In [ ]:
#!git clone https://github.com/CVEProject/cvelist
#%cd cvelist/
import subprocess 
ls = !ls 2000/0xxx
folders = ['1999', '2000', '2001', '2002', '2003', '2004', '2005', '2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018', '2019', '2020', '2021']
print()
cvelist = []
for folder in folders:
    ls = "ls " + folder
    ls = subprocess.getstatusoutput(ls)
    print(ls)
    str_list = ls[1].split('\n')
    #str_list[:] = [x for x in str_list if x]
    print(str_list)
    for f in str_list:
        ls = "ls " + folder + '/' + f
        ls = subprocess.getstatusoutput(ls)
        print(ls[1])
        jsonlist = ls[1].split('\n')
        jsonlist[:] = [x for x in jsonlist if x]
        print(jsonlist)
        for j in jsonlist:
            current_file = open(folder + '/' + f + '/' + j)
            current = json.load(current_file)
            cvelist.append(current)
            print('Inserted element ' + j)
print(len(cvelist))
print(cvelist[0])
    #cvelist.append(json.load(folder))

In [ ]:
%cd ..

Save to json

In [ ]:
with open('cvelist.json', 'w') as fout:
    json.dump(cvelist, fout)

In [ ]:
!ls

Load json

In [ ]:
with open('cvelist.json', 'r') as fin:
    aaaa = json.load(fin)

In [ ]:
print(str(len(cvelist)) + ' ' + str(len(aaaa)))